In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, ensemble, tree
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV,cross_validate
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import joblib
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")


import theano
import theano.tensor as Tensor
import random
import sys
import csv
import time
import utils
import visualize
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, Flatten 

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data5 = pd.read_csv('data5.csv')

In [3]:
train,test= train_test_split(data5, test_size=0.3,random_state=123, stratify=data5.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [4]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Tensorflow/Keras: 2.9.0


In [5]:
X_test = np.expand_dims(X_test, axis=2)

In [6]:
X_test.shape

(314573, 52, 1)

In [7]:
X_train = np.expand_dims(X_train, axis=2)

In [8]:
X_train.shape

(734002, 52, 1)

In [9]:
model = keras.Sequential()
# GRU LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
model.add(GRU(50, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
model.add(BatchNormalization())
model.add(GRU(30, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(GRU(15, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
# ---------------------------------------------------------------------------------------------------
# OPTIMIZER SETTINGS
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

# MODEL COMPILE
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','Recall','Precision'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 52, 50)            7950      
                                                                 
 dropout (Dropout)           (None, 52, 50)            0         
                                                                 
 batch_normalization (BatchN  (None, 52, 50)           200       
 ormalization)                                                   
                                                                 
 gru_1 (GRU)                 (None, 52, 30)            7380      
                                                                 
 dropout_1 (Dropout)         (None, 52, 30)            0         
                                                                 
 batch_normalization_1 (Batc  (None, 52, 30)           120       
 hNormalization)                                        

In [10]:
# RUN THE MODEL
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, verbose=1,
                    validation_data=(X_test, Y_test))

Epoch 1/10
11469/11469 [==============================] - 869s 75ms/step - loss: 0.4638 - accuracy: 0.8002 - recall: 0.0501 - precision: 0.4491 - val_loss: 0.4261 - val_accuracy: 0.8069 - val_recall: 0.1375 - val_precision: 0.5445
Epoch 2/10
11469/11469 [==============================] - 867s 76ms/step - loss: 0.4111 - accuracy: 0.8169 - recall: 0.2045 - precision: 0.6092 - val_loss: 0.3776 - val_accuracy: 0.8333 - val_recall: 0.3874 - val_precision: 0.6259
Epoch 3/10
11469/11469 [==============================] - 901s 79ms/step - loss: 0.3694 - accuracy: 0.8381 - recall: 0.3481 - precision: 0.6750 - val_loss: 0.3456 - val_accuracy: 0.8486 - val_recall: 0.3383 - val_precision: 0.7638
Epoch 4/10
11469/11469 [==============================] - 887s 77ms/step - loss: 0.3400 - accuracy: 0.8521 - recall: 0.4250 - precision: 0.7096 - val_loss: 0.3358 - val_accuracy: 0.8557 - val_recall: 0.4332 - val_precision: 0.7259
Epoch 5/10
11469/11469 [==============================] - 875s 76ms/step - l